In [3]:
import pandas as pd
import numpy as np


In [4]:

df = pd.read_excel(
    'online_retail.xlsx',
    engine="openpyxl"
)


In [5]:
df['Country'].value_counts()


Country
United Kingdom          356728
Germany                   9480
France                    8475
EIRE                      7475
Spain                     2528
Netherlands               2371
Belgium                   2069
Switzerland               1877
Portugal                  1471
Australia                 1258
Norway                    1086
Italy                      803
Channel Islands            757
Finland                    695
Cyprus                     611
Sweden                     461
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
USA                        291
Israel                     247
Unspecified                241
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon                     45


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401604 entries, 0 to 401603
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   InvoiceNo      401604 non-null  object        
 1   StockCode      401604 non-null  object        
 2   Description    401604 non-null  object        
 3   Quantity       401604 non-null  int64         
 4   InvoiceDate    401604 non-null  datetime64[ns]
 5   UnitPrice      401604 non-null  float64       
 6   CustomerID     401604 non-null  int64         
 7   Country        401604 non-null  object        
 8   quantity_flag  401604 non-null  object        
 9   price_flag     401604 non-null  object        
 10  invoice_flag   401604 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(7)
memory usage: 33.7+ MB


In [7]:
df["CustomerID"].isna().sum()


np.int64(0)

In [8]:
df['Quantity'].unique()

array([     6,      8,      2,     32,      3,      4,     24,     12,
           48,     18,     20,     36,     80,     64,     10,    120,
           96,     23,      5,      1,     -1,     50,     40,    100,
          192,    432,    144,    288,    -12,    -24,     16,      9,
          128,     25,     30,     28,      7,     72,    200,    600,
          480,     -6,     14,     -2,     -4,     -5,     -7,     -3,
           11,     70,    252,     60,    216,    384,     27,    108,
           52,  -9360,     75,    270,     42,    240,     90,    320,
           17,   1824,    204,     69,    -36,   -192,   -144,    160,
         2880,   1400,     19,     39,    -48,    -50,     56,     13,
         1440,     -8,     15,    720,    -20,    156,    324,     41,
          -10,    -72,    -11,    402,    378,    150,    300,     22,
           34,    408,    972,    208,   1008,     26,   1000,    -25,
         1488,    250,   1394,    400,    110,    -14,     37,    -33,
      

In [9]:
df['Description'].unique()


array(['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN',
       'CREAM CUPID HEARTS COAT HANGER', ...,
       'PINK CRYSTAL SKULL PHONE CHARM',
       'CREAM HANGING HEART T-LIGHT HOLDER',
       'PAPER CRAFT , LITTLE BIRDIE'], shape=(3896,), dtype=object)

In [10]:
df_returns = df[df["Quantity"] < 0]
df_sales = df[df["Quantity"] >0]


In [11]:
df_sales.shape

(392732, 11)

In [12]:
df_sales = df_sales[df_sales["UnitPrice"] > 0]


In [13]:
df_sales["Revenue"] = df_sales["Quantity"] * df_sales["UnitPrice"]


In [14]:
df_sales["CustomerID"] = df_sales["CustomerID"].astype(int)
df_sales["InvoiceDate"] = pd.to_datetime(df_sales["InvoiceDate"])


In [15]:
import pandas as pd

pd.set_option("display.max_rows", None)


In [16]:
df_sales.drop({'invoice_flag', 'price_flag','quantity_flag' }, axis=1, inplace=True)

In [17]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392692 entries, 0 to 401603
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    392692 non-null  object        
 1   StockCode    392692 non-null  object        
 2   Description  392692 non-null  object        
 3   Quantity     392692 non-null  int64         
 4   InvoiceDate  392692 non-null  datetime64[ns]
 5   UnitPrice    392692 non-null  float64       
 6   CustomerID   392692 non-null  int64         
 7   Country      392692 non-null  object        
 8   Revenue      392692 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 30.0+ MB


In [18]:
df_sales.to_csv("clean_sales.csv", index=False)
df_returns.to_csv("returns.csv", index=False)


In [19]:
df_sales = pd.read_csv("clean_sales.csv", parse_dates=["InvoiceDate"])
df_returns = pd.read_csv("returns.csv", parse_dates=["InvoiceDate"])


In [20]:
cutoff_date = pd.Timestamp("2011-10-01")


In [21]:
sales_obs = df_sales[df_sales["InvoiceDate"] < cutoff_date]
sales_future = df_sales[df_sales["InvoiceDate"] >= cutoff_date]


In [22]:
returns_obs = df_returns[df_returns["InvoiceDate"] < cutoff_date]


In [23]:
future_customers = set(sales_future["CustomerID"].unique())

customer_churn = (
    sales_obs[["CustomerID"]]
    .drop_duplicates()
    .assign(
        Churn=lambda x: (~x["CustomerID"].isin(future_customers)).astype(int)
    )
)


In [24]:
df_sales["InvoiceDate"].min(), df_sales["InvoiceDate"].max()


(Timestamp('2010-12-01 08:26:00'), Timestamp('2011-12-09 12:50:00'))

In [25]:
customer_churn.shape



(3616, 2)

In [26]:
snapshot_date = sales_obs["InvoiceDate"].max() + pd.Timedelta(days=1)


In [27]:
sales_obs = sales_obs.copy()
sales_obs["TotalPrice"] = sales_obs["Quantity"] * sales_obs["UnitPrice"]


In [28]:
rfm = (
    sales_obs
    .groupby("CustomerID")
    .agg(
        Recency=("InvoiceDate", lambda x: (snapshot_date - x.max()).days),
        Frequency=("InvoiceNo", "nunique"),
        Monetary=("TotalPrice", "sum"),
        Tenure=("InvoiceDate", lambda x: (x.max() - x.min()).days)
    )
    .reset_index()
)


In [29]:
returns_agg = (
    returns_obs
    .groupby("CustomerID")["Quantity"]
    .sum()
    .abs()
    .reset_index(name="ReturnedQty")
)

sales_qty = (
    sales_obs
    .groupby("CustomerID")["Quantity"]
    .sum()
    .reset_index(name="SoldQty")
)

return_rate = sales_qty.merge(
    returns_agg, on="CustomerID", how="left"
).fillna(0)

return_rate["ReturnRate"] = (
    return_rate["ReturnedQty"] / return_rate["SoldQty"]
).clip(0, 1)


In [30]:
customer_features = (
    rfm
    .merge(return_rate[["CustomerID", "ReturnRate"]], on="CustomerID", how="left")
    .merge(customer_churn, on="CustomerID", how="left")
)


In [31]:
customer_features["Churn"].value_counts()

Churn
0    1838
1    1778
Name: count, dtype: int64

MODELLING


In [32]:
X = customer_features.drop(columns=["CustomerID", "Churn"])
y = customer_features["Churn"]


In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y
)


In [34]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [35]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    random_state=42
)

log_reg.fit(X_train_scaled, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [36]:
y_pred = log_reg.predict(X_test_scaled)
y_prob = log_reg.predict_proba(X_test_scaled)[:, 1]


In [37]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

confusion_matrix(y_test, y_pred)

array([[292, 168],
       [126, 318]])

In [38]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.70      0.63      0.67       460
           1       0.65      0.72      0.68       444

    accuracy                           0.67       904
   macro avg       0.68      0.68      0.67       904
weighted avg       0.68      0.67      0.67       904



In [39]:
roc_auc_score(y_test, y_prob)

0.7554494712103408

In [40]:
coef_df = pd.DataFrame({
    "Feature": X.columns,
    "Coefficient": log_reg.coef_[0]
}).sort_values(by="Coefficient", ascending=False)

coef_df


,Feature,Coefficient
0,Recency,0.263434
4,ReturnRate,0.048297
2,Monetary,-0.011287
3,Tenure,-0.137626
1,Frequency,-1.471698


In [41]:
import numpy as np
from sklearn.metrics import classification_report

thresholds = np.arange(0.3, 0.8, 0.05)

for t in thresholds:
    y_pred_t = (y_prob >= t).astype(int)
    report = classification_report(y_test, y_pred_t, output_dict=True)
    recall_churn = report['1']['recall']
    precision_churn = report['1']['precision']
    print(
        f"Threshold {t:.2f} | "
        f"Recall (Churn): {recall_churn:.2f} | "
        f"Precision (Churn): {precision_churn:.2f}"
    )


Threshold 0.30 | Recall (Churn): 0.95 | Precision (Churn): 0.58
Threshold 0.35 | Recall (Churn): 0.91 | Precision (Churn): 0.60
Threshold 0.40 | Recall (Churn): 0.85 | Precision (Churn): 0.62
Threshold 0.45 | Recall (Churn): 0.79 | Precision (Churn): 0.64
Threshold 0.50 | Recall (Churn): 0.72 | Precision (Churn): 0.65
Threshold 0.55 | Recall (Churn): 0.65 | Precision (Churn): 0.68
Threshold 0.60 | Recall (Churn): 0.54 | Precision (Churn): 0.73
Threshold 0.65 | Recall (Churn): 0.41 | Precision (Churn): 0.76
Threshold 0.70 | Recall (Churn): 0.27 | Precision (Churn): 0.80
Threshold 0.75 | Recall (Churn): 0.09 | Precision (Churn): 0.75


In [42]:
best_threshold = 0.40
y_pred_final = (y_prob >= best_threshold).astype(int)

from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(y_test, y_pred_final)
print(classification_report(y_test, y_pred_final))


              precision    recall  f1-score   support

           0       0.77      0.49      0.60       460
           1       0.62      0.85      0.71       444

    accuracy                           0.67       904
   macro avg       0.69      0.67      0.66       904
weighted avg       0.70      0.67      0.66       904



In [43]:
X_all = customer_features.drop(columns=["CustomerID", "Churn"])

customer_features["Churn_Probability"] = log_reg.predict_proba(
    scaler.transform(X_all)
)[:, 1]


In [44]:
def retention_action(p):
    if p >= 0.80:
        return "High Risk – Personal Call + 30% Discount"
    elif p >= 0.60:
        return "Medium Risk – 15% Discount Email"
    elif p >= 0.40:
        return "Low Risk – Engagement Campaign"
    else:
        return "No Action"

customer_features["Retention_Action"] = (
    customer_features["Churn_Probability"].apply(retention_action)
)


In [45]:
customer_features["Retention_Action"].value_counts()


Retention_Action
Medium Risk – 15% Discount Email            1354
Low Risk – Engagement Campaign              1195
No Action                                   1063
High Risk – Personal Call + 30% Discount       4
Name: count, dtype: int64

In [46]:
customer_features[
    ["CustomerID", "Churn_Probability", "Retention_Action"]
].head(10)


,CustomerID,Churn_Probability,Retention_Action
0,12346,0.822214,High Risk – Personal Call + 30% Discount
1,12347,0.294118,No Action
2,12348,0.303081,No Action
3,12350,0.739069,Medium Risk – 15% Discount Email
4,12352,0.187436,No Action
5,12353,0.672947,Medium Risk – 15% Discount Email
6,12354,0.691589,Medium Risk – 15% Discount Email
7,12355,0.680089,Medium Risk – 15% Discount Email
8,12356,0.616417,Medium Risk – 15% Discount Email
9,12358,0.636673,Medium Risk – 15% Discount Email


In [47]:
customer_features.head(5)

,CustomerID,Recency,Frequency,Monetary,Tenure,ReturnRate,Churn,Churn_Probability,Retention_Action
0,12346,256,1,77183.60,0,1.000000,1,0.822214,High Risk – Personal Call + 30% Discount
1,12347,60,5,2790.86,237,0.000000,0,0.294118,No Action
2,12348,6,4,1797.24,282,0.000000,1,0.303081,No Action
3,12350,240,1,334.40,0,0.000000,1,0.739069,Medium Risk – 15% Discount Email
4,12352,3,7,2194.31,224,0.161369,0,0.187436,No Action


In [48]:
customer_features.to_csv("retail_analysis.csv", index=False)